# ConvNets

In [1]:
import sys
sys.path.insert(0,'..')
sys.path.insert(0,'../data generation')
sys.path.insert(0,'../model architecture')

In [2]:
from data_generator import load_dataset
from models import *
from experiment_tools import *
from params import *

Using backend: pytorch


## Dataset
### Set hyper-parameters: dataset

In [3]:
test_cases, dataset_size, std_scaler, test_type = get_test_params()

### Select a dataset

In [4]:
idx = 0
test_case = test_cases[idx] 
test_case

'pglib_opf_case24_ieee_rts.m'

### Load a dataset

In [5]:
dataset = load_dataset(test_case=test_case,
                       dataset_size=dataset_size,
                       std_scaler=std_scaler,
                       test_type=test_type,
                       gen_only=True)

### Check the shape of the dataset
- number of samples
- number of buses
- number of generators

## Model
### Set hyper-parameters: model

In [6]:
class ConvNet(nn.Module):
    def __init__(self, output_dim):
        super(ConvNet, self).__init__()
        self.conv1d_1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=5)
        self.conv1d_2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5)
        self.conv1d_3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5)
        self.pooling = nn.MaxPool1d(kernel_size=4, stride=2)
        self.dense_1 = nn.Linear(576, 512)
        self.dense_2 = nn.Linear(512, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # 1st conv layer
        x = F.relu(self.conv1d_1(x.unsqueeze_(1)))
        x = self.pooling(x)
        # 2nd conv layer
        x = F.relu(self.conv1d_2(x))
        x = self.pooling(x)
        # 3rd conv layer
        x = F.relu(self.conv1d_3(x))
        # reshape
        x = x.view(x.size(0), -1)
        # dense layer
        x = F.relu(self.dense_1(x))
        x = self.dense_2(x)
        x = self.sigmoid(x)
        return x

In [7]:
model_name = "conv_net"
num_epochs, batch_size, lr = get_h_params(model_name)

### Set data-loaders

In [8]:
train_loader, val_loader, test_loader = data_prep(dataset=dataset,
                                                  batch_size=batch_size)

### Check the shape of the data-loaders
- number of batch
- number of buses
- number of generators

In [9]:
feature, label = next(iter(train_loader))
feature.shape, label.shape

(torch.Size([32, 24]), torch.Size([32, 66]))

In [10]:
feature, label = next(iter(val_loader))
feature.shape, label.shape

(torch.Size([32, 24]), torch.Size([32, 66]))

In [11]:
feature, label = next(iter(test_loader))
feature.shape, label.shape

(torch.Size([1, 24]), torch.Size([1, 66]))

### Set a model

In [12]:
conv_net = ConvNet(dataset['y'].shape[1])

# ???
loss_func = nn.BCELoss()
# loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(conv_net.parameters(), lr=lr, momentum=0.9)

conv_net.eval()

ConvNet(
  (conv1d_1): Conv1d(1, 16, kernel_size=(5,), stride=(1,))
  (conv1d_2): Conv1d(16, 32, kernel_size=(5,), stride=(1,))
  (conv1d_3): Conv1d(32, 64, kernel_size=(5,), stride=(1,))
  (pooling): MaxPool1d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dense_1): Linear(in_features=576, out_features=512, bias=True)
  (dense_2): Linear(in_features=512, out_features=66, bias=True)
  (sigmoid): Sigmoid()
)

### Train the model

In [13]:
train(net=conv_net,
      train_loader=train_loader,
      val_loader=val_loader,
      optimizer=optimizer,
      loss_func=loss_func,
      model_name=model_name,
      dataset_name=test_case,
      num_epochs=num_epochs)

> case: pglib_opf_case24_ieee_rts
> model: conv_net
- num of params: 342274
- training


RuntimeError: Calculated padded input size per channel: (1). Kernel size: (5). Kernel size can't be greater than actual input size

### Test the model

In [ ]:
test(net=conv_net, test_loader=test_loader, model_name="conv_net", dataset_name=test_case)